In [14]:
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.restoration import denoise_wavelet
import os
from scipy import stats
from tqdm import tqdm
import joblib

class SensorClassifier:
    def __init__(self):
        self.svm = SVC(kernel='rbf', C=10, gamma=0.1)
        self.feature_names = [
            'row_corr_mean', 'row_corr_median', 'row_corr_mode', 'row_corr_max', 'row_corr_min',
            'row_corr_var', 'row_corr_skew', 'row_corr_kurtosis',
            'col_corr_mean', 'col_corr_median', 'col_corr_mode', 'col_corr_max', 'col_corr_min',
            'col_corr_var', 'col_corr_skew', 'col_corr_kurtosis',
            'row_col_ratio'
        ]
    
    def extract_noise(self, image):
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        denoised = denoise_wavelet(image, method='BayesShrink', mode='soft', rescale_sigma=True)
        noise = image.astype(np.float32) - denoised * 255
        return noise

    def compute_correlations(self, noise):
        M, N = noise.shape
        row_ref = np.mean(noise, axis=0)
        col_ref = np.mean(noise, axis=1)
        
        row_corr = [np.corrcoef(noise[i, :], row_ref)[0, 1] if not np.isnan(np.corrcoef(noise[i, :], row_ref)[0, 1]) else 0 for i in range(M)]
        col_corr = [np.corrcoef(noise[:, j], col_ref)[0, 1] if not np.isnan(np.corrcoef(noise[:, j], col_ref)[0, 1]) else 0 for j in range(N)]
        
        return np.array(row_corr), np.array(col_corr)

    def extract_features(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image: {image_path}")
        noise = self.extract_noise(image)
        row_corr, col_corr = self.compute_correlations(noise)
        features = [
            np.mean(row_corr), np.median(row_corr), stats.mode(row_corr, keepdims=False)[0],
            np.max(row_corr), np.min(row_corr),
            np.var(row_corr), stats.skew(row_corr), stats.kurtosis(row_corr),
            np.mean(col_corr), np.median(col_corr), stats.mode(col_corr, keepdims=False)[0],
            np.max(col_corr), np.min(col_corr),
            np.var(col_corr), stats.skew(col_corr), stats.kurtosis(col_corr),
            np.mean(row_corr) / (np.mean(col_corr) + 1e-10)
        ]
        return np.array(features)

    def train(self, image_paths, labels):
        print("Extracting features...")
        X = []
        for path in tqdm(image_paths):
            try:
                features = self.extract_features(path)
                X.append(features)
            except Exception as e:
                print(f"Skipping {path}: {e}")
        
        X = np.array(X)
        y = np.array(labels[:len(X)])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        print("Training SVM...")
        self.svm.fit(X_train, y_train)
        val_pred = self.svm.predict(X_val)
        accuracy = accuracy_score(y_val, val_pred)
        print(f"Validation Accuracy: {accuracy:.2f}")

    def predict(self, image_path):
        features = self.extract_features(image_path)
        return self.svm.predict([features])[0]

    def save_model(self, path):
        joblib.dump(self.svm, path)

    def load_model(self, path):
        self.svm = joblib.load(path)


In [18]:
if __name__ == "__main__":
    # Initialize classifier
    classifier = SensorClassifier()
    
    # Set proper paths
    camera_dir = "images/camera"
    scanner_dir = "images/scanner"
    
    # Collect full image paths
    camera_images = [os.path.join(camera_dir, img) for img in os.listdir(camera_dir) if img.endswith((".jpg", ".png", ".jpeg"))]
    scanner_images = [os.path.join(scanner_dir, img) for img in os.listdir(scanner_dir) if img.endswith((".jpg", ".png", ".jpeg"))]
    
    image_paths = camera_images + scanner_images
    labels = [0]*len(camera_images) + [1]*len(scanner_images)
    
    if len(image_paths) == 0:
        print("No images found for training. Please check the folders.")
        exit()
    
    print(f"Loaded {len(camera_images)} camera images and {len(scanner_images)} scanner images.")
    
    # Train the classifier
    classifier.train(image_paths, labels)
    
    # Test prediction (replace with a real image path)
    test_image = "test_image.png"
    if os.path.exists(test_image):
        prediction = classifier.predict(test_image)
        result = "Scanner" if prediction == 1 else "Camera"
        print(f"The image {test_image} was classified as: {result}")
    else:
        print(f"Test image {test_image} not found.")
        

    for img_path in image_paths:
        if os.path.exists(img_path):
            prediction = classifier.predict(img_path)
            result = "Scanner" if prediction == 1 else "Camera"
            print(f"The image {img_path} was classified as: {result}")
        else:
            print(f"Image {img_path} not found.")


Loaded 3 camera images and 1 scanner images.
Extracting features...


 25%|██▌       | 1/4 [00:02<00:07,  2.49s/it]/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
100%|██████████| 4/4 [00:03<00:00,  1.11it/s]


Training SVM...
Validation Accuracy: 0.00
The image test_image.png was classified as: Camera
The image images/camera/WhatsApp Image 2025-03-19 at 7.09.13 PM.jpeg was classified as: Camera


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


The image images/camera/12241390_10_5.jpeg was classified as: Scanner


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


The image images/camera/12241390_10_3.jpeg was classified as: Camera
The image images/scanner/12241390_10_5.jpeg was classified as: Scanner


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
camera_images = os.listdir("images/camera")
scanner_images = os.listdir("images/scanner")

['requriements.txt', 'env', 'images', '1.ipynb', '1.py']

In [19]:
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.restoration import denoise_wavelet
import os
from scipy import stats
from tqdm import tqdm
import joblib

class SensorClassifier:
    def __init__(self):
        self.svm = SVC(kernel='rbf', C=10, gamma=0.1)
        self.feature_names = [
            'row_corr_mean', 'row_corr_median', 'row_corr_mode', 'row_corr_max', 'row_corr_min',
            'row_corr_var', 'row_corr_skew', 'row_corr_kurtosis',
            'col_corr_mean', 'col_corr_median', 'col_corr_mode', 'col_corr_max', 'col_corr_min',
            'col_corr_var', 'col_corr_skew', 'col_corr_kurtosis',
            'row_col_ratio'
        ]
    
    def extract_noise(self, image):
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        denoised = denoise_wavelet(image, method='BayesShrink', mode='soft', rescale_sigma=True)
        noise = image.astype(np.float32) - denoised * 255
        return noise

    def compute_correlations(self, noise):
        M, N = noise.shape
        row_ref = np.mean(noise, axis=0)
        col_ref = np.mean(noise, axis=1)
        
        row_corr = [np.corrcoef(noise[i, :], row_ref)[0, 1] if not np.isnan(np.corrcoef(noise[i, :], row_ref)[0, 1]) else 0 for i in range(M)]
        col_corr = [np.corrcoef(noise[:, j], col_ref)[0, 1] if not np.isnan(np.corrcoef(noise[:, j], col_ref)[0, 1]) else 0 for j in range(N)]
        
        return np.array(row_corr), np.array(col_corr)

    def extract_features(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image: {image_path}")
        noise = self.extract_noise(image)
        row_corr, col_corr = self.compute_correlations(noise)
        features = [
            np.mean(row_corr), np.median(row_corr), stats.mode(row_corr, keepdims=False)[0],
            np.max(row_corr), np.min(row_corr),
            np.var(row_corr), stats.skew(row_corr), stats.kurtosis(row_corr),
            np.mean(col_corr), np.median(col_corr), stats.mode(col_corr, keepdims=False)[0],
            np.max(col_corr), np.min(col_corr),
            np.var(col_corr), stats.skew(col_corr), stats.kurtosis(col_corr),
            np.mean(row_corr) / (np.mean(col_corr) + 1e-10)
        ]
        return np.array(features)

    def train(self, image_paths, labels):
        print("Extracting features...")
        X = []
        for path in tqdm(image_paths):
            try:
                features = self.extract_features(path)
                X.append(features)
            except Exception as e:
                print(f"Skipping {path}: {e}")
        
        X = np.array(X)
        y = np.array(labels[:len(X)])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        print("Training SVM...")
        self.svm.fit(X_train, y_train)
        val_pred = self.svm.predict(X_val)
        accuracy = accuracy_score(y_val, val_pred)
        print(f"Validation Accuracy: {accuracy:.2f}")

    def predict(self, image_path):
        features = self.extract_features(image_path)
        return self.svm.predict([features])[0]

    def save_model(self, path):
        joblib.dump(self.svm, path)

    def load_model(self, path):
        self.svm = joblib.load(path)


if __name__ == "__main__":
    # Initialize classifier
    classifier = SensorClassifier()
    
    # Set proper paths
    camera_dir = "images/camera"
    scanner_dir = "images/scanner"
    
    # Collect full image paths
    camera_images = [os.path.join(camera_dir, img) for img in os.listdir(camera_dir) if img.endswith((".jpg", ".png", ".jpeg"))]
    scanner_images = [os.path.join(scanner_dir, img) for img in os.listdir(scanner_dir) if img.endswith((".jpg", ".png", ".jpeg"))]
    
    image_paths = camera_images + scanner_images
    labels = [0]*len(camera_images) + [1]*len(scanner_images)
    
    if len(image_paths) == 0:
        print("No images found for training. Please check the folders.")
        exit()
    
    print(f"Loaded {len(camera_images)} camera images and {len(scanner_images)} scanner images.")
    
    # Train the classifier
    classifier.train(image_paths, labels)
    
    # Test prediction (replace with a real image path)
    test_image = "test_image.png"
    if os.path.exists(test_image):
        prediction = classifier.predict(test_image)
        result = "Scanner" if prediction == 1 else "Camera"
        print(f"The image {test_image} was classified as: {result}")
    else:
        print(f"Test image {test_image} not found.")
        
    # Predict for all images in the training set
    for img_path in image_paths:
        if os.path.exists(img_path):
            prediction = classifier.predict(img_path)
            result = "Scanner" if prediction == 1 else "Camera"
            print(f"The image {img_path} was classified as: {result}")
        else:
            print(f"Image {img_path} not found.")


Loaded 3 camera images and 1 scanner images.
Extracting features...


 25%|██▌       | 1/4 [00:01<00:04,  1.53s/it]/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
100%|██████████| 4/4 [00:02<00:00,  1.88it/s]


Training SVM...
Validation Accuracy: 0.00
The image test_image.png was classified as: Camera
The image images/camera/WhatsApp Image 2025-03-19 at 7.09.13 PM.jpeg was classified as: Camera


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


The image images/camera/12241390_10_5.jpeg was classified as: Scanner


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


The image images/camera/12241390_10_3.jpeg was classified as: Camera
The image images/scanner/12241390_10_5.jpeg was classified as: Scanner


In [20]:
features = classifier.extract_features(image_paths[0])
print(f"Extracted features: {features}")


Extracted features: [ 1.69647113e-02  1.67560689e-02 -8.41866707e-02  1.07864603e-01
 -8.41866707e-02  7.60040597e-04 -1.23427343e-02 -1.29831504e-01
  1.64950488e-02  1.62803251e-02 -5.50194826e-02  8.57689126e-02
 -5.50194826e-02  5.01764374e-04  3.39830428e-02 -8.30206085e-02
  1.02847294e+00]


In [22]:
from sklearn.preprocessing import StandardScaler

class SensorClassifier:
    def __init__(self):
        self.svm = SVC(kernel='rbf', C=10, gamma=0.1)
        self.scaler = StandardScaler()  # Add scaler
        self.feature_names = [
            'row_corr_mean', 'row_corr_median', 'row_corr_mode', 'row_corr_max', 'row_corr_min',
            'row_corr_var', 'row_corr_skew', 'row_corr_kurtosis',
            'col_corr_mean', 'col_corr_median', 'col_corr_mode', 'col_corr_max', 'col_corr_min',
            'col_corr_var', 'col_corr_skew', 'col_corr_kurtosis',
            'row_col_ratio'
        ]
    
    def train(self, image_paths, labels):
        print("Extracting features...")
        X = []
        for path in tqdm(image_paths):
            try:
                features = self.extract_features(path)
                X.append(features)
            except Exception as e:
                print(f"Skipping {path}: {e}")
        
        X = np.array(X)
        y = np.array(labels[:len(X)])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Scale the features
        X_train = self.scaler.fit_transform(X_train)
        X_val = self.scaler.transform(X_val)
        
        print("Training SVM...")
        self.svm.fit(X_train, y_train)
        val_pred = self.svm.predict(X_val)
        accuracy = accuracy_score(y_val, val_pred)
        print(f"Validation Accuracy: {accuracy:.2f}")


In [25]:
svm = SVC(kernel='rbf', C=10, gamma=0.1, class_weight='balanced')


In [26]:
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from skimage.restoration import denoise_wavelet
from scipy import stats
from tqdm import tqdm
import os
import joblib
from sklearn.preprocessing import StandardScaler

class SensorClassifier:
    def __init__(self):
        self.svm = SVC(kernel='rbf', C=10, gamma=0.1, class_weight='balanced')  # Add class weight
        self.scaler = StandardScaler()  # Scaler for feature normalization
        self.feature_names = [
            'row_corr_mean', 'row_corr_median', 'row_corr_mode', 'row_corr_max', 'row_corr_min',
            'row_corr_var', 'row_corr_skew', 'row_corr_kurtosis',
            'col_corr_mean', 'col_corr_median', 'col_corr_mode', 'col_corr_max', 'col_corr_min',
            'col_corr_var', 'col_corr_skew', 'col_corr_kurtosis',
            'row_col_ratio'
        ]
    
    def extract_noise(self, image):
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        denoised = denoise_wavelet(image, method='BayesShrink', mode='soft', rescale_sigma=True)
        noise = image.astype(np.float32) - denoised * 255
        return noise

    def compute_correlations(self, noise):
        M, N = noise.shape
        row_ref = np.mean(noise, axis=0)
        col_ref = np.mean(noise, axis=1)
        
        row_corr = [np.corrcoef(noise[i, :], row_ref)[0, 1] if not np.isnan(np.corrcoef(noise[i, :], row_ref)[0, 1]) else 0 for i in range(M)]
        col_corr = [np.corrcoef(noise[:, j], col_ref)[0, 1] if not np.isnan(np.corrcoef(noise[:, j], col_ref)[0, 1]) else 0 for j in range(N)]
        
        return np.array(row_corr), np.array(col_corr)

    def extract_features(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image: {image_path}")
        noise = self.extract_noise(image)
        row_corr, col_corr = self.compute_correlations(noise)
        features = [
            np.mean(row_corr), np.median(row_corr), stats.mode(row_corr, keepdims=False)[0],
            np.max(row_corr), np.min(row_corr),
            np.var(row_corr), stats.skew(row_corr), stats.kurtosis(row_corr),
            np.mean(col_corr), np.median(col_corr), stats.mode(col_corr, keepdims=False)[0],
            np.max(col_corr), np.min(col_corr),
            np.var(col_corr), stats.skew(col_corr), stats.kurtosis(col_corr),
            np.mean(row_corr) / (np.mean(col_corr) + 1e-10)
        ]
        return np.array(features)

    def train(self, image_paths, labels):
        print("Extracting features...")
        X = []
        for path in tqdm(image_paths):
            try:
                features = self.extract_features(path)
                X.append(features)
            except Exception as e:
                print(f"Skipping {path}: {e}")
        
        X = np.array(X)
        y = np.array(labels[:len(X)])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Scale the features
        X_train = self.scaler.fit_transform(X_train)
        X_val = self.scaler.transform(X_val)
        
        print("Training SVM...")
        self.svm.fit(X_train, y_train)
        val_pred = self.svm.predict(X_val)
        accuracy = accuracy_score(y_val, val_pred)
        precision = precision_score(y_val, val_pred)
        recall = recall_score(y_val, val_pred)
        f1 = f1_score(y_val, val_pred)

        print(f"Validation Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("Classification Report:\n", classification_report(y_val, val_pred))

    def predict(self, image_path):
        features = self.extract_features(image_path)
        features_scaled = self.scaler.transform([features])  # Scale the test feature
        return self.svm.predict(features_scaled)[0]

    def save_model(self, path):
        joblib.dump(self.svm, path)

    def load_model(self, path):
        self.svm = joblib.load(path)


if __name__ == "__main__":
    # Initialize classifier
    classifier = SensorClassifier()
    
    # Set proper paths
    camera_dir = "images/camera"
    scanner_dir = "images/scanner"
    
    # Collect full image paths
    camera_images = [os.path.join(camera_dir, img) for img in os.listdir(camera_dir) if img.endswith((".jpg", ".png", ".jpeg"))]
    scanner_images = [os.path.join(scanner_dir, img) for img in os.listdir(scanner_dir) if img.endswith((".jpg", ".png", ".jpeg"))]
    
    image_paths = camera_images + scanner_images
    labels = [0]*len(camera_images) + [1]*len(scanner_images)
    
    if len(image_paths) == 0:
        print("No images found for training. Please check the folders.")
        exit()
    
    print(f"Loaded {len(camera_images)} camera images and {len(scanner_images)} scanner images.")
    
    # Train the classifier
    classifier.train(image_paths, labels)
    
    # Test prediction (replace with a real image path)
    test_image = "test_image.png"
    if os.path.exists(test_image):
        prediction = classifier.predict(test_image)
        result = "Scanner" if prediction == 1 else "Camera"
        print(f"The image {test_image} was classified as: {result}")
    else:
        print(f"Test image {test_image} not found.")
        
    # Predict for all images in dataset
    for img_path in image_paths:
        if os.path.exists(img_path):
            prediction = classifier.predict(img_path)
            result = "Scanner" if prediction == 1 else "Camera"
            print(f"The image {img_path} was classified as: {result}")
        else:
            print(f"Image {img_path} not found.")


Loaded 3 camera images and 1 scanner images.
Extracting features...


 25%|██▌       | 1/4 [00:01<00:04,  1.59s/it]/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
100%|██████████| 4/4 [00:02<00:00,  1.78it/s]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to

Training SVM...
Validation Accuracy: 0.00
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

The image test_image.png was classified as: Camera
The image images/camera/WhatsApp Image 2025-03-19 at 7.09.13 PM.jpeg was classified as: Camera
The image images/camera/12241390_10_5.jpeg was classified as: Scanner


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


The image images/camera/12241390_10_3.jpeg was classified as: Camera
The image images/scanner/12241390_10_5.jpeg was classified as: Scanner


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [27]:
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from skimage.restoration import denoise_wavelet
from scipy import stats
from tqdm import tqdm
import os
import joblib
from sklearn.preprocessing import StandardScaler

class SensorClassifier:
    def __init__(self):
        self.svm = SVC(kernel='rbf', C=10, gamma=0.1, class_weight='balanced')  # Add class weight
        self.scaler = StandardScaler()  # Scaler for feature normalization
        self.feature_names = [
            'row_corr_mean', 'row_corr_median', 'row_corr_mode', 'row_corr_max', 'row_corr_min',
            'row_corr_var', 'row_corr_skew', 'row_corr_kurtosis',
            'col_corr_mean', 'col_corr_median', 'col_corr_mode', 'col_corr_max', 'col_corr_min',
            'col_corr_var', 'col_corr_skew', 'col_corr_kurtosis',
            'row_col_ratio'
        ]
    
    def extract_noise(self, image):
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        denoised = denoise_wavelet(image, method='BayesShrink', mode='soft', rescale_sigma=True)
        noise = image.astype(np.float32) - denoised * 255
        return noise

    def compute_correlations(self, noise):
        M, N = noise.shape
        row_ref = np.mean(noise, axis=0)
        col_ref = np.mean(noise, axis=1)
        
        row_corr = [np.corrcoef(noise[i, :], row_ref)[0, 1] if not np.isnan(np.corrcoef(noise[i, :], row_ref)[0, 1]) else 0 for i in range(M)]
        col_corr = [np.corrcoef(noise[:, j], col_ref)[0, 1] if not np.isnan(np.corrcoef(noise[:, j], col_ref)[0, 1]) else 0 for j in range(N)]
        
        return np.array(row_corr), np.array(col_corr)

    def extract_features(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image: {image_path}")
        noise = self.extract_noise(image)
        row_corr, col_corr = self.compute_correlations(noise)
        
        # Ensure that the features do not contain NaN values
        features = [
            np.mean(row_corr), np.median(row_corr), stats.mode(row_corr, keepdims=False)[0],
            np.max(row_corr), np.min(row_corr),
            np.var(row_corr), stats.skew(row_corr), stats.kurtosis(row_corr),
            np.mean(col_corr), np.median(col_corr), stats.mode(col_corr, keepdims=False)[0],
            np.max(col_corr), np.min(col_corr),
            np.var(col_corr), stats.skew(col_corr), stats.kurtosis(col_corr),
            np.mean(row_corr) / (np.mean(col_corr) + 1e-10)  # Add small constant to prevent division by zero
        ]
        
        # Ensure no NaN or infinite values are present
        features = [f if np.isfinite(f) else 0 for f in features]
        return np.array(features)

    def train(self, image_paths, labels):
        print("Extracting features...")
        X = []
        for path in tqdm(image_paths):
            try:
                features = self.extract_features(path)
                X.append(features)
            except Exception as e:
                print(f"Skipping {path}: {e}")
        
        X = np.array(X)
        y = np.array(labels[:len(X)])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Scale the features
        X_train = self.scaler.fit_transform(X_train)
        X_val = self.scaler.transform(X_val)
        
        print("Training SVM...")
        self.svm.fit(X_train, y_train)
        val_pred = self.svm.predict(X_val)
        accuracy = accuracy_score(y_val, val_pred)
        precision = precision_score(y_val, val_pred, zero_division=1)
        recall = recall_score(y_val, val_pred, zero_division=1)
        f1 = f1_score(y_val, val_pred, zero_division=1)

        print(f"Validation Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("Classification Report:\n", classification_report(y_val, val_pred))

    def predict(self, image_path):
        features = self.extract_features(image_path)
        features_scaled = self.scaler.transform([features])  # Scale the test feature
        return self.svm.predict(features_scaled)[0]

    def save_model(self, path):
        joblib.dump(self.svm, path)

    def load_model(self, path):
        self.svm = joblib.load(path)


if __name__ == "__main__":
    # Initialize classifier
    classifier = SensorClassifier()
    
    # Set proper paths
    camera_dir = "images/camera"
    scanner_dir = "images/scanner"
    
    # Collect full image paths
    camera_images = [os.path.join(camera_dir, img) for img in os.listdir(camera_dir) if img.endswith((".jpg", ".png", ".jpeg"))]
    scanner_images = [os.path.join(scanner_dir, img) for img in os.listdir(scanner_dir) if img.endswith((".jpg", ".png", ".jpeg"))]
    
    image_paths = camera_images + scanner_images
    labels = [0]*len(camera_images) + [1]*len(scanner_images)
    
    if len(image_paths) == 0:
        print("No images found for training. Please check the folders.")
        exit()
    
    print(f"Loaded {len(camera_images)} camera images and {len(scanner_images)} scanner images.")
    
    # Train the classifier
    classifier.train(image_paths, labels)
    
    # Test prediction (replace with a real image path)
    test_image = "test_image.png"
    if os.path.exists(test_image):
        prediction = classifier.predict(test_image)
        result = "Scanner" if prediction == 1 else "Camera"
        print(f"The image {test_image} was classified as: {result}")
    else:
        print(f"Test image {test_image} not found.")
        
    # Predict for all images in dataset
    for img_path in image_paths:
        if os.path.exists(img_path):
            prediction = classifier.predict(img_path)
            result = "Scanner" if prediction == 1 else "Camera"
            print(f"The image {img_path} was classified as: {result}")
        else:
            print(f"Image {img_path} not found.")


Loaded 3 camera images and 1 scanner images.
Extracting features...


 25%|██▌       | 1/4 [00:01<00:04,  1.56s/it]/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
100%|██████████| 4/4 [00:02<00:00,  1.83it/s]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` para

Training SVM...
Validation Accuracy: 0.00
Precision: 0.00
Recall: 1.00
F1 Score: 0.00
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

The image test_image.png was classified as: Camera
The image images/camera/WhatsApp Image 2025-03-19 at 7.09.13 PM.jpeg was classified as: Camera
The image images/camera/12241390_10_5.jpeg was classified as: Scanner


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


The image images/camera/12241390_10_3.jpeg was classified as: Camera
The image images/scanner/12241390_10_5.jpeg was classified as: Scanner


/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/navadeep/Desktop/DIP/env/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
